 Example.jl


 This file imports the data from csv files with Hoechle's data and then
 runs the regressions.

 Compare the results to Table 2 in Hoechle, 2007, "Robust Standard Errors...",
 Stata Journal, 7, 281-312.
 
 Paul.Soderlind@unisg.ch   Apr 2016

In [6]:
using Printf, DelimitedFiles, LinearAlgebra, Statistics

include("jlFiles/FindNoNaNPs.jl")
include("jlFiles/HDirProdPs.jl")
include("jlFiles/iterationPrintPs.jl")
include("jlFiles/HszDk5dwPs.jl")

HszDk5dwPs (generic function with 6 methods)

In [7]:
#using JLD
#xTN = load("bidaskspread.jld","xTN")          #load data, from Hoechle
#(BA,TRMS,TRMS2,aVol,Size)    = [xTN[:,:,i] for i=4:8]

BA    = readdlm("Data/BA.csv",',')
TRMS  = readdlm("Data/TRMS.csv",',')
TRMS2 = readdlm("Data/TRMS2.csv",',')
aVol  = readdlm("Data/aVol.csv",',')
Size  = readdlm("Data/Size.csv",',')

(T,N) = size(BA)

(61, 219)

In [8]:
println("\nPooled OLS")
x   = [vec(aVol') vec(Size') vec(TRMS2') vec(TRMS') vec(ones(T,N)')]
y   = vec(BA')           #stack rows in column vector

vv  = FindNoNaNPs(1,y,x)
(y,x) = (y[vv],x[vv,:])    #prune NaNs

b    = x\y
res  = y - x*b
Covb = inv(x'x)*var(res)
Stdb = sqrt.(diag(Covb))
println("b, Std and tstat from OLS")
display(round.([b Stdb b./Stdb],digits=4))


Pooled OLS
b, Std and tstat from OLS


5×3 Matrix{Float64}:
 -0.0018  0.0004   -4.007
 -0.1519  0.0087  -17.4151
  0.0033  0.0006    5.2959
 -0.0018  0.005    -0.3698
  1.4591  0.0577   25.2702

In [9]:
println("\nHszDk5dwPs, effectively weighting all obs equally")
x = ones(T)
z = cat(aVol,Size,TRMS2,TRMS,ones(T,N),dims=3)
fnO = HszDk5dwPs(BA,x,z,false,8,0)
theta    = fnO.theta
stdDKj   = fnO.stdDKj

println("theta, std and t-stat")
display(round.([theta stdDKj theta./stdDKj],digits=4))

println("Compare the results to Table 2 in Hoechle, 2007, 'Robust Standard Errors...'")


HszDk5dwPs, effectively weighting all obs equally
theta, std and t-stat


5×3 Matrix{Float64}:
 -0.0018  0.0011   -1.6267
 -0.1519  0.0103  -14.7892
  0.0033  0.0009    3.7726
 -0.0018  0.0052   -0.3508
  1.4591  0.1354   10.7749

Compare the results to Table 2 in Hoechle, 2007, 'Robust Standard Errors...'


In [10]:
println("\nHszDk5dwPs, effectively weighting all periods equally, irrespective of number of obs")
x = ones(T,1)
z = cat(aVol,Size,TRMS2,TRMS,ones(T,N),dims=3)
fnO = HszDk5dwPs(BA,x,z,false,8,1)
theta    = fnO.theta
stdDKj   = fnO.stdDKj

println("theta, std and t-stat")
display(round.([theta stdDKj theta./stdDKj],digits=4))


HszDk5dwPs, effectively weighting all periods equally, irrespective of number of obs
theta, std and t-stat


5×3 Matrix{Float64}:
 -0.0021  0.0011   -1.8878
 -0.1497  0.0098  -15.2201
  0.0031  0.0009    3.4814
 -0.0007  0.006    -0.1093
  1.465   0.1407   10.4109

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*